In [1]:
import torch
import copy

from torch import nn
from algorithms.iterative_relaxation import IterativeRelaxation
from algorithms.decision_procedure import MarabouCoreDP
from models.test_models import ProphecyPaperNetwork, TestModel
from models.utils import attach_relu_activation_hook, attach_layer_output_hook, get_layers_info

Instructions for updating:
non-resource variables are not supported in the long term


### Test DP

In [2]:
dp = MarabouCoreDP()
prophecy_model = ProphecyPaperNetwork()

In [8]:
activation = {
  'linear_relu_stack.1': ['ON', 'OFF'], 
  'linear_relu_stack.3': ['ON', 'OFF'],
}
dp.solve(activation, prophecy_model, 0)

['unsat',
 {},

In [9]:
activation = {
  'linear_relu_stack.1': ['ON', 'OFF'], 
  'linear_relu_stack.3': ['--', '--'],
}
dp.solve(activation, prophecy_model, 0)

['unsat',
 {},

In [5]:
activation = {
  'linear_relu_stack.1': ['--', '--'], 
  'linear_relu_stack.3': ['ON', '--'],
}
dp.solve(activation, prophecy_model, 0)

['sat',
 {0: -49.999999,
  1: -50.000001,
  2: 1.9999999949504854e-06,
  3: -100.0,
  4: 1.9999999949504854e-06,
  5: 0.0,
  6: 9.999999974752427e-07,
  7: -9.999999974752427e-07,
  8: 9.999999974752427e-07,
  9: 9.999999974752427e-07,
  10: -0.0,
  11: -0.0},

### Test Iterative Relaxation

In [10]:
ir_model = ProphecyPaperNetwork()
iterative_relaxation = IterativeRelaxation()

input_property = iterative_relaxation.call(ir_model, [[1, -1]], 0)
print(f"input property: {input_property}")

input property: [{'linear_relu_stack.1': ['ON', 'OFF'], 'linear_relu_stack.3': ['--', '--']}]


In [11]:
ir_model = ProphecyPaperNetwork()
iterative_relaxation = IterativeRelaxation()

input_property = iterative_relaxation.call(ir_model, [[1, -1]], 1)
print(f"input property: {input_property}")

input property: [{'linear_relu_stack.1': ['ON', 'OFF'], 'linear_relu_stack.3': ['ON', 'OFF']}, 1]


### Misc

In [13]:
model = ProphecyPaperNetwork()
_act_handles, activation_signature = attach_relu_activation_hook(model)
_out_handles, layer_outputs = attach_layer_output_hook(model)

# input_data = [[-1,3]]
input_data = [[1, -1]]
# input_data = [[-10, -0]]

X = torch.tensor(input_data, dtype=torch.float)
logits = model(X)
prediction_prob = nn.Softmax(dim=1)(logits)
y_pred = prediction_prob.argmax(1)

print(f"Predicted class: {y_pred}")
print(f"Activation: {activation_signature}")
print(f"Layer outputs: {layer_outputs}")

Predicted class: tensor([0])
Activation: {'linear_relu_stack.1': ['ON', 'OFF'], 'linear_relu_stack.3': ['ON', 'OFF']}
Layer outputs: {'linear_relu_stack.0': tensor([[2., 0.]]), 'linear_relu_stack.1': tensor([[2., 0.]]), 'linear_relu_stack.2': tensor([[ 1., -1.]]), 'linear_relu_stack.3': tensor([[1., 0.]]), 'final_output': tensor([[ 1., -1.]])}
